In [26]:
# CONNECT
from qdrant_client import QdrantClient,models
client=QdrantClient(url="http://localhost:6333")

In [4]:
'''
ubuntu
docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    qdrant/qdrant
'''

'\nubuntu\ndocker run -p 6333:6333 -p 6334:6334     -v $(pwd)/qdrant_storage:/qdrant/storage:z     qdrant/qdrant\n'

In [2]:
'''
window:
docker run -p 6333:6333 -v /absolute/path/here/qdrant_storage:/qdrant/storage qdrant/qdrant
'''

'\nwindow:\ndocker run -p 6333:6333 -v /absolute/path/here/qdrant_storage:/qdrant/storage qdrant/qdrant\n'

In [27]:
my_collection="SimeCSE_Vn_doan"

In [28]:
#TẠO COLLECTION

client.delete_collection(collection_name=my_collection)
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
)



True

In [29]:
import torch
from collecting_data_sql import data_sql
dataset=data_sql()
from SimeCSE_Vn_model import embedding_text


client.upload_points(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=idx,
            vector=embedding_text(doc['noi-dung']),
            payload=doc,
        )for idx,doc in enumerate(dataset)
    ],
)




torch.Size([1, 32])
['Công_bố', 'kèm', 'theo', 'Quyết_định', 'này', 'thủ_tục', 'hành_chính', 'nội_bộ', 'mới', 'ban_hành', 'và', 'sửa_đổi', 'bổ_sung', 'trong', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'giữa', 'các', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'và', 'Du_lịch']
torch.Size([1, 11])
['Quyết_định', 'này', 'có', 'hiệu_lực', 'thi_hành', 'kể', 'từ', 'ngày', 'ký']
torch.Size([1, 88])
['Thủ_tục', 'hành_chính', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'có', 'số', 'thứ', 'tự', '01', '02', 'điểm', 'A1', 'mục', 'A', 'danh_mục', '1', 'ban_hành', 'kèm', 'theo', 'Quyết_định', 'số', '7@@', '86', 'QĐ', 'B@@', 'VHTTDL', 'ngày', '31', 'tháng', '3', 'năm', '2023', 'của', 'Bộ_trưởng', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'và', 'Du_lịch', 'về', 'việc', 'công_bố', 'thủ_tục', 'hành_chính', 'nội_bộ', 'giữa', 'các', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'và', 'trong', 'nội_bộ', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'thuộc', 'phạm_vi', 'chức_năng', 'q

SEARCH

In [22]:
# search toàn bộ văn bản
def search_full(query_vector, limit,my_collection):
   my_collection=my_collection
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit
   )
   for i in search_result:
      print(i)


# chỉ in ra 1 số thông tin cần thiết

def search_with_some_inf(query_vector, limit, my_collection):
   
   my_collection=my_collection
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      # query_filter=models.Filter(
      #    must=[models.FieldCondition(key="dieu", match=models.MatchValue(value="2"))]
      # ),
      limit=limit,
      search_params=models.SearchParams(
               exact=True,  # Turns on the exact search mode KNN
            ),
   )

   #print question
   if len(search_result)==0:
      print("Không có thông tin tìm kiếm")
   else:
      print("Kết quả thông tin tìm kiếm là:")
      answer_array=[]
      for i in search_result:

         answer_json={
            'index':i.id,
            'score':i.score,
            'noi dung':i.payload['noi-dung'], ################## noi dung
            'id_van_ban':i.payload['id-van-ban'],
            'so-hieu':i.payload['so-hieu'],
            # 'loai-van-ban':i.payload['loai-van-ban'],
            # 'noi-ban-hanh':i.payload['noi-ban-hanh'],
            # 'nguoi-ky':i.payload['nguoi-ky'],
            # 'ngay-ban-hanh':i.payload['ngay-ban-hanh'],
            # 'ngay-hieu-luc': i.payload['ngay-hieu-luc'],
            # 'ngay-cong-bao': i.payload['ngay-cong-bao'],
            # 'so-cong-bao':i.payload['so-cong-bao'],
            # 'tinh-trang':i.payload['tinh-trang'],
            'chuong':i.payload['chuong'],
            'muc':i.payload['muc'],
            'dieu':i.payload['dieu'],
     
         }
         if len(answer_array)==0:
            answer_array.append(answer_json)
         else:
            count=0
            for answer in answer_array:
            #id là của 1 văn bản , phải thêm chương, điều, mục
               if (answer_json['index']==answer['index']):
                  count=count+1
            if count==0:
               answer_array.append(answer_json)

     
      
      for i in answer_array:
         print(i)



In [23]:
import torch
from SimeCSE_Vn_model import embedding_text
text=' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịch'
my_collection='SimeCSE_Vn_doan'
search_full(embedding_text(text),3,my_collection)


torch.Size([1, 32])
['Công_bố', 'kèm', 'theo', 'Quyết_định', 'này', 'thủ_tục', 'hành_chính', 'nội_bộ', 'mới', 'ban_hành', 'và', 'sửa_đổi', 'bổ_sung', 'trong', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'giữa', 'các', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'và', 'Du_lịch']
id=0 version=0 score=1.0 payload={'chuong': 0, 'dieu': 1, 'id-van-ban': 1121, 'loai-van-ban': 'Quyết định', 'muc': 0, 'ngay-ban-hanh': '11/04/2024', 'ngay-cong-bao': 'Đang cập nhật', 'ngay-hieu-luc': '11/04/2024', 'nguoi-ky': 'Tạ Quang Đông', 'noi-ban-hanh': 'Bộ Văn hoá, Thể thao và du lịch', 'noi-dung': ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịch', 'so-cong-bao': 'Đang cập nhật', 'so-hieu': '987/QĐ-BVHTTDL', 'tinh-trang': 'Còn hiệu lực'} vector=None shard_key=None
id=449 version=7 score=0.9573444 payload={'chuong': 0, 'dieu': 1, 

In [24]:
import torch
from SimeCSE_Vn_model import embedding_text
text = ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịchh'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),10,my_collection)


torch.Size([1, 36])
['Công_bố', 'kèm', 'theo', 'Quyết_định', 'này', 'thủ_tục', 'hành_chính', 'nội_bộ', 'mới', 'ban_hành', 'và', 'sửa_đổi', 'bổ_sung', 'trong', 'lĩnh_vực', 'Thi_đua', 'khen_thưởng', 'giữa', 'các', 'cơ_quan', 'đơn_vị', 'trực_thuộc', 'Bộ', 'Văn_@@', 'h@@', 'ó@@', 'a', 'Thể_thao', 'và', 'D@@', 'u_l@@', 'ị@@', 'ch@@', 'h']
Kết quả thông tin tìm kiếm là:
{'index': 0, 'score': 0.97110337, 'noi dung': ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng giữa các cơ quan  đơn vị trực thuộc Bộ Văn hóa  Thể thao và Du lịch', 'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 1}
{'index': 449, 'score': 0.92570883, 'noi dung': ' Công bố kèm theo Quyết định này thủ tục hành chính nội bộ giữa các cơ quan hành chính nhà nước mới ban hành và sửa đổi  bổ sung trong lĩnh vực Thi đua  khen thưởng thuộc phạm vi chức năng quản lý của Bộ Văn hóa  Thể thao và Du lịch', 'id_van_ban': 1122, '

In [25]:
import torch
from SimeCSE_Vn_model import embedding_text
text = '  Vụ Tổ chức cán bộ thẩm định hồ sơ khen thưởng và trình Hội đồng Thi đua  khen thưởng Bộ trong thời hạn 25 ngày làm việc kể từ ngày nhận đủ hồ sơ theo quy định (trừ khen thưởng đột xuất) \
Đối với trường hợp hồ sơ không đầy đủ  trong thời hạn 05 ngày làm việc  Vụ Tổ chức cán bộ thông báo tới cơ quan  đơn vị trình hoàn thiện hồ sơ'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),20,my_collection)


torch.Size([1, 52])
['Vụ', 'Tổ_chức', 'cán_bộ', 'thẩm_định', 'hồ_sơ', 'khen_thưởng', 'và', 'trình', 'Hội_đồng', 'Thi_đua', 'khen_thưởng', 'Bộ', 'trong', 'thời_hạn', '25', 'ngày', 'làm_việc', 'kể', 'từ', 'ngày', 'nhận', 'đủ', 'hồ_sơ', 'theo', 'quy_định', '(', 'trừ', 'khen_thưởng', 'đột_xuất', ')', 'Đối_với', 'trường_hợp', 'hồ_sơ', 'không', 'đầy_đủ', 'trong', 'thời_hạn', '05', 'ngày', 'làm_việc', 'Vụ', 'Tổ_chức', 'cán_bộ', 'thông_báo', 'tới', 'cơ_quan', 'đơn_vị', 'trình', 'hoàn_thiện', 'hồ_sơ']
Kết quả thông tin tìm kiếm là:
{'index': 10, 'score': 1.0, 'noi dung': '  Vụ Tổ chức cán bộ thẩm định hồ sơ khen thưởng và trình Hội đồng Thi đua  khen thưởng Bộ trong thời hạn 25 ngày làm việc kể từ ngày nhận đủ hồ sơ theo quy định (trừ khen thưởng đột xuất) Đối với trường hợp hồ sơ không đầy đủ  trong thời hạn 05 ngày làm việc  Vụ Tổ chức cán bộ thông báo tới cơ quan  đơn vị trình hoàn thiện hồ sơ', 'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 97, 

In [13]:
import torch
from SimeCSE_Vn_model import embedding_text
text = ' Các trường hợp đề nghị khen thưởng trên đã được xét duyệt  thẩm định đảm bảo thế nào'

my_collection='SimeCSE_Vn_doan'
answer_array=search_with_some_inf(embedding_text(text),10,my_collection)


torch.Size([1, 14])
['Các', 'trường_@@', 'hợp_@@', 'đề_nghị', 'khen_thưởng', 'trên', 'đã', 'được', 'xét_duyệt', 'thẩm_định', 'đảm_bảo', 'thế_nào']
Kết quả thông tin tìm kiếm là:
{'index': 491, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu': '988/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 587, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu': '988/QĐ-BVHTTDL', 'chuong': 0, 'muc': 0, 'dieu': 3}
{'index': 649, 'score': 0.89580476, 'noi dung': 'Các trường hợp đề nghị khen thưởng trên đã được 1 xét duyệt  thẩm định đảm bảo đúng quy định về đối tượng  tiêu chuẩn  hồ sơ  thủ tục theo quy định hiện hành', 'id_van_ban': 1122, 'so-hieu'